## Function decorators and closures

In [2]:
@decorate
def target():
    print('running target()')
    
# Has the same effect as writing this:

def target():
    print('running target()')

target = decorate(target)

NameError: name 'decorate' is not defined

In [3]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')
    
target()

running inner()


In [4]:
target

<function __main__.deco.<locals>.inner()>

In [5]:
# registration.py
# When Python executes decorators
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()

running register(<function f1 at 0x1042050d0>)
running register(<function f2 at 0x104205488>)
running main()
registry -> [<function f1 at 0x1042050d0>, <function f2 at 0x104205488>]
running f1()
running f2()
running f3()


### Closures 闭包

In [9]:
# average_oo.py
# a class to calculate a running average
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

avg = Averager()
avg(10)

10.0

In [10]:
avg(11)

10.5

In [11]:
avg(12)

11.0

In [13]:
# average.py 
# a higher-order function to calculate a running average
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

avg = make_averager()
avg(10)

10.0

In [14]:
avg(11)

10.5

In [15]:
avg.__code__.co_varnames

('new_value', 'total')

In [16]:
avg.__code__.co_freevars

('series',)

In [17]:
avg.__closure__

(<cell at 0x10417c258: list object at 0x1041e8b88>,)

In [21]:
avg.__closure__[0].cell_contents

[10, 11]

In [22]:
# The nonlocal declaration
def make_averager():
    count = 0
    total = 0
    
    def averager(new_Value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

### Implementing a simple decorator

In [ ]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__